In [1]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas
#!pip install ipywidgets

import re
import unidecode
from spellchecker import SpellChecker
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import datadetox as detox
import importlib
importlib.reload(detox)

<module 'datadetox' from 'c:\\Users\\USER\\Desktop\\GitHub\\Brasileirao-Analise\\3. Source Code\\Transformed\\datadetox.py'>

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [25]:
pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 8):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

#fitragem série B comjumto 7
df_list[6] = df_list[6][df_list[6]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {  'data': 'Data','Date': 'Data','data_partida': 'Data',
                        'time_mandante': 'Mandante','mandante': 'Mandante','Home': 'Mandante',
                        'time_visitante': 'Visitante','visitante': 'Visitante','Away': 'Visitante'}
    

df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

df_list = detox.padronizar_datas(df_list)


DataFrame 0: Já é datetime64, mantido como está.
DataFrame 1: Convertendo padrão com '/'
DataFrame 2: Convertendo padrão com '/'
DataFrame 3: Convertendo padrão com '/'
DataFrame 4: Já é datetime64, mantido como está.
DataFrame 5: Já é datetime64, mantido como está.
DataFrame 6: Convertendo padrão ISO com '-'


Unir todos os DFs com agrupamento por colunas, Data, Mandante e Visitante

In [26]:
#mapear cluber qeu o filtro BERT nao resolve (manualmente)
clubes_map_inicial = {  "America-RN":"América Natal","América-RN":"América Natal","América rn":"América Natal",
                        "Joinvile":"Joinville",
                        "Atlético pr":"Athlético-PR","Atlético-PR":"Athlético-PR", 
                        "Grêmio Prudente":"Barueri","Grêmio prudente":"Barueri","Gremio Prudente":"Barueri"}

for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_map_inicial)
    df['Visitante'] = df['Visitante'].replace(clubes_map_inicial)

# listar clubes Mandantes e visitantes, para agrupar ortograficamente
clubes_unicos = []
for df in df_list:
    unicos_por_bd = pd.unique(np.concatenate([df['Mandante'].values, df['Visitante'].values])).tolist()
    clubes_unicos = list(set(clubes_unicos) | set(unicos_por_bd))

clubes_value_dict = [   "América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                        "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                        "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                        "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                        "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                        "Sport Recife", "Vasco da Gama", "Vitória"]

modelo = SentenceTransformer('all-MiniLM-L6-v2')
clubes_corrigidos = detox.corrigir_times_semanticamente(clubes_unicos, clubes_value_dict,modelo)
clubes_dict = detox.gerar_dicionario_mapeamento(clubes_corrigidos, limite=0.0)

# corrigindo e agrupando clubes em todos os dataframes
for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_dict)
    df['Visitante'] = df['Visitante'].replace(clubes_dict)

# unindo todos os dfs com merge on=['Data','Mandante','Visitante']
df_merged = pd.DataFrame(columns=['Data','Mandante','Visitante'])
for i,df in enumerate(df_list):
    df_merged = pd.merge(df_merged, df, how='outer', on=['Data','Mandante','Visitante'],suffixes=('', f'_{i}'))


In [31]:
import datetime as dt
df = df_merged.copy()

# Criar edições dos campeonatos visto que 2020 foi ate 2021
mask = (df['Data'] > dt.datetime(2020, 8, 1)) & (df['Data'] < dt.datetime(2021, 2, 28))
df.loc[mask, 'edition'] = 2020
df.loc[~mask, 'edition'] = df.loc[~mask, 'Data'].dt.year

#print(df['edition'].value_counts())

df = df[df['edition'] == 2009]
# duplicados = df[df.duplicated(subset=['Mandante', 'Visitante'], keep=False)]

#duplicados.sort_values(by=['Mandante', 'Visitante'], ascending=[True, False])
#display(duplicados.sort_values(by='Mandante'))





In [32]:
def separar_df_agrupado(df, chaves):
    return [c for c in df.columns if c not in chaves]

def mesclar_grupo(grupo, colunas_dados):
    for col in colunas_dados:
        valores_unicos = grupo[col].dropna().unique()
        if len(valores_unicos) > 1:
            return None
    #return grupo.ffill().bfill().iloc[0]
    return grupo.ffill().bfill().infer_objects(copy=False).iloc[0]

In [33]:
chaves = ['Data', 'Mandante', 'Visitante', 'edition']
colunas_dados = separar_df_agrupado(df, chaves)
grupo_dup = df.groupby(chaves, as_index=False)
linhas_mescladas = grupo_dup.apply(lambda g: mesclar_grupo(g, colunas_dados))
linhas_mescladas = linhas_mescladas.dropna(how='any').reset_index(drop=True)

display(linhas_mescladas)

C:\Users\USER\AppData\Local\Temp\ipykernel_14360\822675371.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return grupo.ffill().bfill().infer_objects(copy=False).iloc[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_14360\464780323.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  linhas_mescladas = grupo_dup.apply(lambda g: mesclar_grupo(g, colunas_dados))


,Data,Mandante,Visitante,Ano,rodada,estadio,arbitro,publico,publico_max,tecnico_mandante,tecnico_visitante,colocacao_mandante,colocacao_visitante,valor_equipe_titular_mandante,valor_equipe_titular_visitante,idade_media_titular_mandante,idade_media_titular_visitante,gols_mandante,gols_visitante,gols_1_tempo_mandante,gols_1_tempo_visitante,escanteios_mandante,escanteios_visitante,faltas_mandante,faltas_visitante,chutes_bola_parada_mandante,chutes_bola_parada_visitante,defesas_mandante,defesas_visitante,impedimentos_mandante,impedimentos_visitante,chutes_mandante,chutes_visitante,chutes_fora_mandante,chutes_fora_visitante,Ano_1,ID,rodata,hora,formacao_mandante,formacao_visitante,tecnico_mandante_1,tecnico_visitante_1,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado,Rodada,Horário,Dia,Vencedor,Arena,Mandante Placar,Visitante Placar,Estado Mandante,Estado Visitante,Estado Vencedor,Ano_3,ID_3,rodata_3,hora_3,formacao_mandante_3,formacao_visitante_3,tecnico_mandante_3,tecnico_visitante_3,vencedor_3,arena_3,mandante_Placar_3,visitante_Placar_3,mandante_Estado_3,visitante_Estado_3,Country,League,Season,Time,HG,AG,Res,PSCH,PSCD,PSCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,BFECH,BFECD,BFECA,resultado,placar_visitante,hora_partida,placar_mandante,estadio_5,mandante_gols_fora_casa,mandante_empates_fora_casa,mandante_total_jogos,mandante_gols_casa,mandante_jogos_fora_casa,mandante_vitorias_casa,mandante_derrotas_casa,mandante_total_pontos,mandante_empates_casa,mandante_pontos_fora_casa,mandante_total_gols_sofridos,mandante_total_vitorias,mandante_vitorias_fora_casa,mandante_total_derrotas,mandante_pontos_casa,mandante_derrotas_fora_casa,mandante_total_gols_marcados,mandante_jogos_casa,mandante_total_empates,visitante_gols_fora_casa,visitante_empates_fora_casa,visitante_total_jogos,visitante_gols_casa,visitante_jogos_fora_casa,visitante_vitorias_casa,visitante_derrotas_casa,visitante_total_pontos,visitante_empates_casa,visitante_pontos_fora_casa,visitante_total_gols_sofridos,visitante_total_vitorias,visitante_vitorias_fora_casa,visitante_total_derrotas,visitante_pontos_casa,visitante_derrotas_fora_casa,visitante_total_gols_marcados,visitante_jogos_casa,visitante_total_empates,rodada_5,ano campeonato,Temporada,Série,Número do Jogo,Rodada_6,Turno,Estádio,Cidade,UF,Dia da Semana,Hora,UF Mandante,Gol Mandante,Gol Visitante,UF Visitante,Gols no Jogo,Resultado,Resultado Mandante,Resultado Visitante,Placar,Link do Jogo,Data de Início da Consulta,Data de Fim da Consulta,edition
